In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
def lorenz(x):
    return np.array([
        sigma * (x[1] - x[0]),
        x[0]*(r - x[2]) - x[1],
        x[0]*x[1] - beta*x[2]
    ])

def jacobian(x):
    return np.array([
        [-sigma, sigma, 0],
        [r - x[2], -1, -x[0]],
        [x[1], x[0], -beta]
    ])


In [3]:
def rk4_step(x, dt):
    k1 = lorenz(x)
    k2 = lorenz(x + 0.5*dt*k1)
    k3 = lorenz(x + 0.5*dt*k2)
    k4 = lorenz(x + dt*k3)
    return x + (dt/6.0)*(k1 + 2*k2 + 2*k3 + k4)

def rk4_variation(x, V, dt):
    J1 = jacobian(x)
    k1 = J1.dot(V)

    x2 = x + 0.5*dt*lorenz(x)
    J2 = jacobian(x2)
    k2 = J2.dot(V + 0.5*dt*k1)

    x3 = x + 0.5*dt*lorenz(x2)
    J3 = jacobian(x3)
    k3 = J3.dot(V + 0.5*dt*k2)

    x4 = x + dt*lorenz(x3)
    J4 = jacobian(x4)
    k4 = J4.dot(V + dt*k3)

    return V + (dt/6.0)*(k1 + 2*k2 + 2*k3 + k4)

In [4]:
dt = 0.005
total_time = 150
orthonorm_interval = 0.05
transient = 50.0
x0 = np.array([1.0, 1.0, 1.0])
n_steps = int(total_time / dt)
steps_per_orth = int(round(orthonorm_interval / dt))

In [5]:
r_ = [27]
sigma_ = [10]
beta_ =  np.linspace(0.2,3.5,180)

In [6]:
if (len(r_) > len(sigma_)) and (len(r_) > len(beta_)):
    sigma_ = sigma_*int((len(r_)/len(sigma_)))
    beta_ = beta_*int((len(r_)/len(beta_)))
elif (len(sigma_) > len(beta_)) and (len(sigma_) > len(r_)):
    r_ = r_*int((len(sigma_)/len(r_)))
    beta_ = beta_*int((len(sigma_)/len(beta_)))
elif (len(beta_) > len(sigma_)) and (len(beta_) > len(r_)):
    r_ = r_*int((len(beta_)/len(r_)))
    sigma_ = sigma_*int((len(beta_)/len(sigma_)))

In [7]:
all_exponents = []
lyap1 = []
lyap2 = []
lyap3 = []
for s in range(0,len(r_)):
    sigma = sigma_[s]
    beta = beta_[s]
    r = r_[s]

    x = x0.copy()
    V = np.eye(3)
    eps = 1e-12
    sums = np.zeros(3)
    count = 0

    traj = []
    times = []
    t = 0.0
    lyap_running = []
    for step in range(n_steps):
        x = rk4_step(x, dt)
        V = rk4_variation(x, V, dt)
        t += dt

        if step % steps_per_orth == 0:
            Q, R = np.linalg.qr(V)
            d = np.diag(R)
            if any (np.abs(d)) < eps:
                d = eps
            signs = np.sign(d)
            signs[signs == 0] = 1
            Q *= signs
            R = (R.T * signs).T

            logs = np.log(np.abs(np.diag(R)) + eps)
            if t > transient:
                sums += logs
                count += 1
                lyap_running.append(sums / (count * orthonorm_interval))

            V = Q.copy()

    lyap = sums / (count * orthonorm_interval)
    lyap_sorted = np.sort(lyap)[::-1]

    all_exponents.append(lyap_sorted)
    lyap1.append(lyap_sorted[0])
    lyap2.append(lyap_sorted[1])
    lyap3.append(lyap_sorted[2])
    if (s+1)%10==0:
        print((s+1)/10)

#plt.scatter()

KeyboardInterrupt: 

In [ ]:
lyap1 = np.array(lyap1)
lyap3 = np.array(lyap3)
lyap2 = np.array(lyap2)

In [ ]:
if r_[0] != r_[-1]:

    fig, axes = plt.subplots(1, 3, figsize=(12, 4))
    axes[0].scatter(r_, lyap1, color='darkred')
    axes[0].set_title('')
    axes[0].set_xlabel('r')
    axes[0].set_ylabel(r'$\lambda_{1}$', rotation=0)
    axes[0].grid(True)

    axes[1].scatter(r_, lyap2, color='mediumblue')
    axes[1].set_title('')
    axes[1].set_xlabel('r')
    axes[1].set_ylabel(r'$\lambda_{2}$', rotation=0)
    axes[1].grid(True)
    plt.title('Lyapunov exponents in dependence of r')

    axes[2].scatter(r_, lyap3, color='seagreen')
    axes[2].set_title('')
    axes[2].set_xlabel('r')
    axes[2].set_ylabel(r'$\lambda_{3}$', rotation = 0)
    axes[2].grid(True)


    plt.tight_layout()
    plt.show()

In [ ]:
if r_[0] != r_[-1]:

    fig, axes = plt.subplots(1, 3, figsize=(12, 4))
    axes[0].scatter(r_, lyap1, color='darkred')
    axes[0].set_title('')
    axes[0].set_xlabel('r')
    axes[0].set_ylabel(r'$\lambda_{1}$', rotation=0)
    axes[0].grid(True)

    axes[1].scatter(r_, lyap2, color='mediumblue')
    axes[1].set_title('')
    axes[1].set_xlabel('r')
    axes[1].set_ylabel(r'$\lambda_{2}$', rotation=0)
    axes[1].grid(True)
    plt.title('Lyapunov exponents in dependence of r')

    axes[2].scatter(r_, lyap3, color='seagreen')
    axes[2].set_title('')
    axes[2].set_xlabel('r')
    axes[2].set_ylabel(r'$\lambda_{3}$', rotation = 0)
    axes[2].grid(True)


    plt.tight_layout()
    plt.show()

In [ ]:
if sigma_[0] != sigma_[-1]:

    fig, axes = plt.subplots(1, 3, figsize=(13, 4))
    axes[0].scatter(sigma_, lyap1, color='darkred')
    axes[0].set_title('')
    axes[0].set_xlabel(r'$\sigma$', rotation=0)
    axes[0].set_ylabel(r'$\lambda_{1}$')
    axes[0].grid(True)

    axes[1].scatter(sigma_, lyap2, color='mediumblue')
    axes[1].set_title('')
    axes[1].set_xlabel(r'$\sigma$')
    axes[1].set_ylabel(r'$\lambda_{2}$', rotation=0)
    axes[1].grid(True)

    axes[2].scatter(sigma_, lyap3, color='seagreen')
    axes[2].set_title('')
    axes[2].set_xlabel(r'$\sigma$')
    axes[2].set_ylabel(r'$\lambda_{3}$', rotation=0)
    axes[2].grid(True)

    plt.tight_layout()
    plt.show()

In [ ]:
if beta_[0] != beta_[-1]:

    fig, axes = plt.subplots(1, 3, figsize=(12, 4))
    axes[0].scatter(beta_, lyap1, color='darkred')
    axes[0].set_title('')
    axes[0].set_xlabel(r'$\beta$')
    axes[0].set_ylabel(r'$\lambda_{1}$',rotation=0)
    axes[0].grid(True)

    axes[1].scatter(beta_, lyap2, color='mediumblue')
    axes[1].set_title('')
    axes[1].set_xlabel(r'$\beta$')
    axes[1].set_ylabel(r'$\lambda_{2}$',rotation=0)
    axes[1].grid(True)

    axes[2].scatter(beta_, lyap3, color='seagreen')
    axes[2].set_title('')
    axes[2].set_xlabel(r'$\beta$')
    axes[2].set_ylabel(r'$\lambda_{3}$',rotation=0)
    axes[2].grid(True)

    plt.tight_layout()
    plt.show()

In [ ]:
def kaplan_yorke_sorted(Lyap1, Lyap2, Lyap3):
    """
    Compute the Kaplan–Yorke dimension from three equally-shaped arrays
    of already-sorted Lyapunov exponents λ1 ≥ λ2 ≥ λ3.
    """
    λ1 = np.array(Lyap1)
    λ2 =  np.array(Lyap2)
    λ3 =  np.array(Lyap3)

    # Cumulative sums
    S1 = λ1
    S2 = λ1 + λ2

    # Initialize result array
    D = np.zeros_like(λ1)

    # Case j = 0 → λ1 < 0  (D=0, already default)
    j1 = S1 >= 0
    j2 = S2 >= 0

    # --- j = 1 -----------------------------------------
    mask1 = j1 & (~j2)
    D[mask1] = 1 + S1[mask1] / np.abs(λ2[mask1])

    # --- j = 2 -----------------------------------------
    mask2 = j2
    # Avoid division by zero:
    zero_mask = (λ3 == 0) & mask2
    nonzero_mask = (λ3 != 0) & mask2

    D[zero_mask] = 2
    D[nonzero_mask] = 2 + S2[nonzero_mask] / np.abs(λ3[nonzero_mask])
    return D
D_yk = (kaplan_yorke_sorted(lyap1, lyap2, lyap3))
plt.scatter(beta_,D_yk, label=r'$\lambda_1$', color='mediumpurple')
plt.xlabel(r'$\sigma$')
plt.ylabel(r'$D_{KY}$', rotation=0)
plt.grid(True)
plt.title('Kaplan-Yorke dimension of the lorenz attractor')
plt.show()
print(D_yk)
print(beta_)